# Detect anagrams in the English dictionary

Two words are anagrams of each other when their letters can be rearranged to turn one word into the other. For instance, **stop** can be anagrammed into **post**, **spot**, **tops**, **pots**, and **opts**. 

To detect anagrams I'll use this simple strategy: 

I will define the **signature** of a word as the sorted list of its letters including duplicates. So the signature of Python would be hnopty. Two words are anagrams of each other if they have the same signature.

Thus, I'm going to make a Python dict of all the words in a dictionary indexed by the signature. Looking up if a word has an anagram would then be as simple as computing its signature and looking it up in the dict.

anagrams_by_signature = {'post':{'post', 'spot', 'tops', 'pots', 'opts'}, ...}

Let's begin!

## Implementation
We begin by loading a dictionary from a file, *words.txt*. The repository contains the 1934 English dictionary that is distributed with many Unix systems.

In Python, we talk of idioms when we think of code constructs that have become the preferred way to achieve a certain goal. One example is looping through all the lines of the text file. In this case we will store the words into a list.

In [2]:
words = []
for line in open('words.txt'):
    words.append(line)

We get more than 200 000 words.

In [3]:
len(words)

235886

In [4]:
words[:10]

['A\n',
 'a\n',
 'aa\n',
 'aal\n',
 'aalii\n',
 'aam\n',
 'Aani\n',
 'aardvark\n',
 'aardwolf\n',
 'Aaron\n']

I do see two problems, though, every word ends in the new line character, and also some words are capitalized, which will interfere with our signature scheme. We can fix both issues using Python string methods.

We can refactor your code to address those issues.

In [5]:
words = []
for line in open('words.txt'):
    words.append(line.strip().lower())

In [6]:
words[:10]

['a',
 'a',
 'aa',
 'aal',
 'aalii',
 'aam',
 'aani',
 'aardvark',
 'aardwolf',
 'aaron']

Problem solved! Ah, I do see a duplicate, which comes from the **A** appearing both in uppercase and lowercase. One way to get rid of that is to build not a list, but a set.

In [7]:
words = set()
for line in open('words.txt'):
    words.add(line.strip().lower())

Given that the body of the loop is just one line, we can do it more idiomatically with a comprehension.

In [8]:
words = {line.strip().lower() for line in open('words.txt')}

Finally, to get the list in alphabetical order, we can just wrap the set in the Python built-in sorted.

In [9]:
words = sorted({line.strip().lower() for line in open('words.txt')})

We are now ready to make anagrams. We need a function to make signatures.

In [10]:
def signature(word):
    return ''.join(sorted(word))

Our strategy is based on building a python dictionary of words indexed by signatures. That is, the keys will be signatures, and the values will be sets of words. We'll call it words_by_signature. We can make it with a loop.

In [11]:
words_by_signautre = {}

for word in words:
    if signature(word) not in words_by_signautre:
        # if we haven't seen this signature, define the dictionary entry with this word
        words_by_signautre[signature(word)] = {word}
    else:
        # otherwise add the word to the set for this signature
        words_by_signautre[signature(word)].add(word)

This works fine, but we should improve on this awkward and repetitious code. The problem is that if a signatory is not already in the dictionary, then we need to create a set with a single word. But if it is there, we need to add a new word to the existing set. We can avoid this complication by using a default dict that will automatically initialize a signature with an empty set. The argument to default dict must be a function that makes the default value. We call it a factory sometimes. So in this case, set will do.If you call it without parameters, you just get the empty set.

In [12]:
import collections

words_by_signature = collections.defaultdict(set)

for word in words:
    words_by_signature[signature(word)].add(word)

We can save some memoryand CPU by removing all signatures that correspond to a single word. Every word is an anagram of itself, but that's not very interesting. A dict comprehension will do the job. Remember to iterate on both key and value, we use dict items.

In [13]:
anagrams_by_signature = {sig: wordset for sig, wordset in words_by_signature.items() if len(wordset) > 1}

So finding the anagrams of python is as simple as looking for its signature in our dictionary.

In [14]:
anagrams_by_signature[signature('python')]

{'phyton', 'python'}

We can do this with a simple function.

In [15]:
def find_anagrams(myword):
    return anagrams_by_signature[signature(myword)]

find_anagrams('tops')

{'post', 'spot', 'stop', 'tops'}

Works fine! What about my name?

In [16]:
find_anagrams('Pedro')

KeyError: 'Pdeor'

So, no anagrams with my name. But perhaps we shouldn't get an error when no anagram is found. How about we just return the empty set? We can do that by catching the exception and doing otherwise.

In [17]:
def find_anagram(myWord):
    try:
        return anagrams_by_signature[signature(myWord)]
    except KeyError:
        return set()

In [18]:
find_anagram('Pedro')

set()

Now that we have set up this machinery, there are many interesting investigations we could do. For instance, we could find the longest anagrams, which we get by sorting the signatures by length. We can use sorted, but we need to tell it to sort by length, not alphabetical order, and to reverse the result.

In [ ]:
sorted(anagrams_by_signature.keys(), key = len, reverse = True)

We can wrap this list into comprehensionto see the actual anagrams.

In [ ]:
[anagrams_by_signature[sig] for sig in sorted(anagrams_by_signature.keys(), key = len, reverse = True)]

How about the set of anagrams with the most words? For this, we sort the dict values instead of the keys.

In [ ]:
sorted(anagrams_by_signature.values(), key=len, reverse=True)

In [ ]:
len(sorted(anagrams_by_signature.values(), key=len, reverse=True)[0])

10